In [2]:
!pip install googletrans

  Preparing metadata (setup.py) ... done
  Using cached chardet-3.0.4-py2.py3-none-any.whl.metadata (3.2 kB)
  Using cached idna-2.10-py2.py3-none-any.whl.metadata (9.1 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 11.4 MB/s eta 0:00:00
  Created wheel for googletrans: filename=googletrans-3.0.0-py3-none-any.whl size=15719 sha256=ca87b63a222d319c2a8f4e752087d6a036cf513756041cb05a0e9fdc8924b230
  Stored in directory: /Users/kwaktaekyeong/Library/Caches/pip/wheels/0e/ce/9b/d51de1064911d42480ab6b57fc943ee36572441f27546354e2
Successfully built googletrans
  Attempting uninstall: h11
    Found existing installation: h11 0.14.0
    Uninstalling h11-0.14.0:
      Successfully uninstalled h11-0.14.0
  Attempting uninstall: idna
    Found existing installation: idna 3.10
    Uninstalling idna-3.10:
      Successfully uninstalled idna-3.10
  Attempting uninstall: httpcore
    Found existing installation: httpcore 1.0.6
    Uninstalling httpcore-1.0.6:
      Successfully uninstalled 

In [5]:
import pandas as pd
import requests
from dotenv import load_dotenv
import os

# .env 파일 로드
load_dotenv()

# Google Cloud Translate API 키 가져오기
api_key = os.getenv("GOOGLE_TRANSLATE_API_KEY")
if not api_key:
    raise ValueError("GOOGLE_TRANSLATE_API_KEY가 .env 파일에 설정되지 않았습니다.")

# 파일 경로
input_file = './grammar_results.csv'
output_file = './translated_grammar_results.csv'

# CSV 파일 로드
df = pd.read_csv(input_file)

# 영어 문장이 들어있는 열 이름 확인
print("데이터프레임 첫 5개 행:\n", df.head())

# 예를 들어, "문장"이라는 열이 번역 대상이라 가정
target_column = "final_sentence"  # 실제 파일에 맞게 수정 필요

# 번역 함수
def translate_text(text, target_language="ko"):
    url = "https://translation.googleapis.com/language/translate/v2"
    params = {
        "q": text,
        "target": target_language,
        "key": api_key,
        "source": "en"
    }
    try:
        response = requests.post(url, data=params)
        if response.status_code == 200:
            translated_text = response.json()["data"]["translations"][0]["translatedText"]
            return translated_text
        else:
            return f"번역 오류: {response.status_code}, {response.text}"
    except Exception as e:
        return f"번역 오류: {e}"

# 새 열 추가 (번역된 문장)
df['sentence_meaning'] = df[target_column].apply(translate_text)

# 번역된 CSV 파일 저장
df.to_csv(output_file, index=False, encoding='utf-8-sig')

print(f"번역 완료. 파일이 저장되었습니다: {output_file}")


데이터프레임 첫 5개 행:
                                  final_sentence  grammar_score  \
0             It’s a knife and it’s not a fork.           0.94   
1             Here are scissors that cut paper.           1.00   
2  Here are scissors that cut paper and knives.           1.00   
3                I have dinner and it's a fork.           1.01   
4                          He can pull the car.           1.01   

   coherence_score      word  
0                1     knife  
1                1  scissors  
2                1       cut  
3                1      fork  
4                1      pull  
번역 완료. 파일이 저장되었습니다: ./translated_grammar_results.csv
